<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [3]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [4]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJFINANCE', 'BERGEPAINT', 'BHARTIARTL',
       'BRITANNIA', 'CANBK', 'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR',
       'DIVISLAB', 'DMART', 'DRREDDY', 'EICHERMOT', 'FEDERALBNK',
       'GODREJCP', 'GODREJIND', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDUNILVR', 'IBREALEST', 'ICICIBANK',
       'ICICIPRULI', 'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTI', 'M&M', 'MARUTI',
       'MCDOWELL-N', 'MINDTREE', 'MOTILALOFS', 'MUTHOOTFIN', 'NATIONALUM',
       'NAUKRI', 'PIDILITIND', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN',
       'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL',
       'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO', 'ZEEL'],
      dtype=object)

In [5]:
mypf['InPortfolio'].value_counts()

0    47
1    14
Name: InPortfolio, dtype: int64

In [6]:
mypf[(mypf['Symbol'] + '.NS') == (stock_ids[1] + '.NS')]['Forecast'].values[0]

1044.0

In [7]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
import talib 
yfin.pdr_override()

In [8]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'EMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = web.DataReader(stock_symbol,  data_source='yahoo', start = start, end = end)['Close']
    stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close Price'].ewm(span = short_window, adjust = False).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close Price'].ewm(span = mid_window, adjust = False).mean(),0)     

    stock_df['RSI'] = talib.RSI(stock_df['Close Price'], timeperiod=14)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close Price'].ewm(span = long_window, adjust = False).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close Price'] - stock_df['200_EMA'])*100/stock_df['200_EMA'],2)
    stock_df['Max'] = round(max(stock_df['Close Price']),0)
    stock_df['Min'] = round(min(stock_df['Close Price']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close Price']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close Price'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Close Price'] = round(stock_df['Close Price'],0) 
    
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    stock_id = stock_id.upper() + '.NS'
    print(stock_id)
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

ASIANPAINT.NS
[*********************100%***********************]  1 of 1 completed
AXISBANK.NS
[*********************100%***********************]  1 of 1 completed
BAJFINANCE.NS
[*********************100%***********************]  1 of 1 completed
BERGEPAINT.NS
[*********************100%***********************]  1 of 1 completed
BHARTIARTL.NS
[*********************100%***********************]  1 of 1 completed
BRITANNIA.NS
[*********************100%***********************]  1 of 1 completed
CANBK.NS
[*********************100%***********************]  1 of 1 completed
CIPLA.NS
[*********************100%***********************]  1 of 1 completed
COLPAL.NS
[*********************100%***********************]  1 of 1 completed
DABUR.NS
[*********************100%***********************]  1 of 1 completed
DEEPAKNTR.NS
[*********************100%***********************]  1 of 1 completed
DIVISLAB.NS
[*********************100%***********************]  1 of 1 completed
DMART.NS
[*******************

In [9]:
stock_prec_dev('^NSEI').tail(15)

[*********************100%***********************]  1 of 1 completed


,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%
Date,,,,,,,,,,,
2022-11-15,18403.0,17960.0,17650.0,68.762083,17162.0,^NSEI,7.23,18812.0,15294.0,88.39,11.61
2022-11-16,18410.0,18003.0,17680.0,68.896962,17175.0,^NSEI,7.19,18812.0,15294.0,88.56,11.44
2022-11-17,18344.0,18035.0,17706.0,65.683868,17186.0,^NSEI,6.74,18812.0,15294.0,86.69,13.31
2022-11-18,18308.0,18061.0,17729.0,63.914098,17197.0,^NSEI,6.46,18812.0,15294.0,85.66,14.34
2022-11-21,18160.0,18071.0,17746.0,57.156610,17207.0,^NSEI,5.54,18812.0,15294.0,81.47,18.53
2022-11-22,18244.0,18087.0,17766.0,59.769463,17217.0,^NSEI,5.97,18812.0,15294.0,83.86,16.14
2022-11-23,18267.0,18104.0,17786.0,60.479618,17228.0,^NSEI,6.03,18812.0,15294.0,84.52,15.48
2022-11-24,18484.0,18141.0,17813.0,66.475077,17240.0,^NSEI,7.22,18812.0,15294.0,90.68,9.32
2022-11-25,18513.0,18176.0,17840.0,67.183433,17253.0,^NSEI,7.30,18812.0,15294.0,91.49,8.51


In [10]:
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close Price'])*100/(df_prec_dev['Close Price']),2)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)

In [11]:
# buy or accumulate (based on std dev from 200 EMA)
df_prec_dev.sort_values(by = 'Dev%_200', ascending=True).head(20)

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-05,669.0,673.0,696.0,41.017441,775.0,MOTILALOFS.NS,-13.72,980.0,654.0,4.49,95.51,930.0,39.01
4,2022-12-05,3369.0,3390.0,3483.0,44.762434,3808.0,DIVISLAB.NS,-11.53,4678.0,3266.0,7.28,92.72,4065.0,20.66
2,2022-12-05,475.0,479.0,499.0,44.113408,527.0,ICICIPRULI.NS,-9.81,616.0,447.0,16.75,83.25,647.0,36.21
35,2022-12-05,412.0,401.0,400.0,66.825741,455.0,WIPRO.NS,-9.37,722.0,375.0,10.76,89.24,433.0,5.10
7,2022-12-05,554.0,559.0,575.0,43.370759,593.0,JUBLFOOD.NS,-6.60,786.0,464.0,27.90,72.10,651.0,17.51
36,2022-12-05,1100.0,1073.0,1061.0,60.743155,1163.0,TECHM.NS,-5.44,1806.0,965.0,16.02,83.98,1156.0,5.09
6,2022-12-05,4056.0,3972.0,3976.0,55.914556,4239.0,NAUKRI.NS,-4.31,5740.0,3375.0,28.81,71.19,4851.0,19.60
23,2022-12-05,88.0,83.0,80.0,63.577452,91.0,IBREALEST.NS,-3.74,179.0,60.0,23.19,76.81,95.0,7.95
15,2022-12-05,80.0,76.0,75.0,67.309351,83.0,NATIONALUM.NS,-3.37,130.0,67.0,20.95,79.05,91.0,13.75
37,2022-12-05,622.0,612.0,614.0,57.150046,643.0,BERGEPAINT.NS,-3.23,791.0,561.0,26.61,73.39,651.0,4.66


In [12]:
# sell or downsize (based on std dev from 200 EMA)
df_prec_dev.sort_values(by = 'Dev%_200', ascending=False).head(15)

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
55,2022-12-05,322.0,312.0,287.0,66.582249,244.0,CANBK.NS,32.17,329.0,175.0,95.78,4.22,310.0,-3.73
21,2022-12-05,137.0,133.0,129.0,61.294405,112.0,FEDERALBNK.NS,22.59,138.0,81.0,98.77,1.23,149.0,8.76
59,2022-12-05,4403.0,4192.0,4010.0,77.281422,3722.0,BRITANNIA.NS,18.31,4433.0,3094.0,97.79,2.21,3935.0,-10.63
9,2022-12-05,899.0,880.0,846.0,65.944904,766.0,AXISBANK.NS,17.39,915.0,626.0,94.53,5.47,1044.0,16.13
13,2022-12-05,617.0,601.0,580.0,66.458166,529.0,SBIN.NS,16.69,617.0,435.0,100.16,-0.16,709.0,14.91
8,2022-12-05,1196.0,1169.0,1153.0,59.464646,1046.0,INDUSINDBK.NS,14.29,1264.0,778.0,85.91,14.09,1394.0,16.56
48,2022-12-05,1039.0,1026.0,994.0,62.126183,912.0,SUNPHARMA.NS,13.97,1052.0,737.0,96.00,4.00,1056.0,1.64
11,2022-12-05,934.0,926.0,906.0,56.512873,827.0,ICICIBANK.NS,12.89,953.0,654.0,93.51,6.49,1075.0,15.10
53,2022-12-05,2086.0,2040.0,1981.0,64.868968,1851.0,LT.NS,12.67,2104.0,1472.0,97.09,2.91,2036.0,-2.40
39,2022-12-05,844.0,837.0,812.0,58.313372,750.0,BHARTIARTL.NS,12.55,851.0,641.0,96.71,3.29,879.0,4.15


In [13]:
# portfolio buy/sell
mypf['Stock'] = mypf['Symbol'] + '.NS'
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)].sort_values(by = 'Dev%_200', ascending=True)

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-05,669.0,673.0,696.0,41.017441,775.0,MOTILALOFS.NS,-13.72,980.0,654.0,4.49,95.51,930.0,39.01
4,2022-12-05,3369.0,3390.0,3483.0,44.762434,3808.0,DIVISLAB.NS,-11.53,4678.0,3266.0,7.28,92.72,4065.0,20.66
2,2022-12-05,475.0,479.0,499.0,44.113408,527.0,ICICIPRULI.NS,-9.81,616.0,447.0,16.75,83.25,647.0,36.21
36,2022-12-05,1100.0,1073.0,1061.0,60.743155,1163.0,TECHM.NS,-5.44,1806.0,965.0,16.02,83.98,1156.0,5.09
6,2022-12-05,4056.0,3972.0,3976.0,55.914556,4239.0,NAUKRI.NS,-4.31,5740.0,3375.0,28.81,71.19,4851.0,19.60
43,2022-12-05,224.0,224.0,225.0,49.788991,227.0,TATAPOWER.NS,-1.23,290.0,195.0,30.74,69.26,232.0,3.57
1,2022-12-05,465.0,436.0,435.0,72.956308,467.0,GODREJIND.NS,-0.50,644.0,405.0,24.96,75.04,638.0,37.20
33,2022-12-05,1142.0,1089.0,1071.0,66.720829,1143.0,MUTHOOTFIN.NS,-0.08,1540.0,956.0,31.87,68.13,1207.0,5.69
50,2022-12-05,3426.0,3348.0,3267.0,65.122662,3322.0,TCS.NS,3.13,4019.0,2982.0,42.81,57.19,3385.0,-1.20
10,2022-12-05,589.0,565.0,553.0,69.671075,568.0,HDFCLIFE.NS,3.68,694.0,504.0,44.68,55.32,683.0,15.96


In [14]:
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%']*100/df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%'].sum()

0     17.353203
1     16.548043
2     16.107651
3     10.106762
4      9.190391
6      8.718861
10     7.099644
14     6.187722
26     3.389680
33     2.531139
36     2.264235
43     1.588078
50    -0.533808
51    -0.551601
Name: Upside%, dtype: float64

In [15]:
# prospects buy
df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)].sort_values(by = 'Dev%_200', ascending=True)

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
35,2022-12-05,412.0,401.0,400.0,66.825741,455.0,WIPRO.NS,-9.37,722.0,375.0,10.76,89.24,433.0,5.10
7,2022-12-05,554.0,559.0,575.0,43.370759,593.0,JUBLFOOD.NS,-6.60,786.0,464.0,27.90,72.10,651.0,17.51
23,2022-12-05,88.0,83.0,80.0,63.577452,91.0,IBREALEST.NS,-3.74,179.0,60.0,23.19,76.81,95.0,7.95
15,2022-12-05,80.0,76.0,75.0,67.309351,83.0,NATIONALUM.NS,-3.37,130.0,67.0,20.95,79.05,91.0,13.75
37,2022-12-05,622.0,612.0,614.0,57.150046,643.0,BERGEPAINT.NS,-3.23,791.0,561.0,26.61,73.39,651.0,4.66
42,2022-12-05,4028.0,4031.0,4125.0,46.414516,4160.0,DMART.NS,-3.17,4821.0,3231.0,50.14,49.86,4177.0,3.70
5,2022-12-05,6660.0,6836.0,6973.0,34.001142,6860.0,BAJFINANCE.NS,-2.91,7853.0,5284.0,53.58,46.42,7978.0,19.79
27,2022-12-05,1234.0,1237.0,1248.0,48.506405,1259.0,HAVELLS.NS,-1.96,1418.0,1070.0,47.23,52.77,1327.0,7.54
46,2022-11-22,3434.0,3497.0,3408.0,47.418459,3500.0,MINDTREE.NS,-1.89,4842.0,2742.0,32.95,67.05,3520.0,2.50
12,2022-12-05,429.0,428.0,425.0,51.912119,436.0,TATAMOTORS.NS,-1.66,525.0,372.0,37.09,62.91,493.0,14.92


In [16]:
# CCI buy
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)].sort_values(by = 'Dev%_200', ascending=True)

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
4,2022-12-05,3369.0,3390.0,3483.0,44.762434,3808.0,DIVISLAB.NS,-11.53,4678.0,3266.0,7.28,92.72,4065.0,20.66
35,2022-12-05,412.0,401.0,400.0,66.825741,455.0,WIPRO.NS,-9.37,722.0,375.0,10.76,89.24,433.0,5.10
7,2022-12-05,554.0,559.0,575.0,43.370759,593.0,JUBLFOOD.NS,-6.60,786.0,464.0,27.90,72.10,651.0,17.51
36,2022-12-05,1100.0,1073.0,1061.0,60.743155,1163.0,TECHM.NS,-5.44,1806.0,965.0,16.02,83.98,1156.0,5.09
37,2022-12-05,622.0,612.0,614.0,57.150046,643.0,BERGEPAINT.NS,-3.23,791.0,561.0,26.61,73.39,651.0,4.66
27,2022-12-05,1234.0,1237.0,1248.0,48.506405,1259.0,HAVELLS.NS,-1.96,1418.0,1070.0,47.23,52.77,1327.0,7.54
46,2022-11-22,3434.0,3497.0,3408.0,47.418459,3500.0,MINDTREE.NS,-1.89,4842.0,2742.0,32.95,67.05,3520.0,2.50
57,2022-11-29,4912.0,4874.0,4774.0,54.013485,4990.0,LTI.NS,-1.56,7562.0,3824.0,29.11,70.89,4615.0,-6.05
29,2022-12-05,3157.0,3133.0,3169.0,52.818013,3152.0,ASIANPAINT.NS,0.16,3576.0,2580.0,57.92,42.08,3361.0,6.46
50,2022-12-05,3426.0,3348.0,3267.0,65.122662,3322.0,TCS.NS,3.13,4019.0,2982.0,42.81,57.19,3385.0,-1.20
